In [335]:
import torch
import numpy as np
import math
import os, sys, glob
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [336]:
path = r'/Users/yixiangsun/Desktop/torch_file'
file = glob.glob(os.path.join(path, "*.pth"))
#print(file[2225])
data = []
for x in file:
    data.append(torch.load(x))

In [337]:
def alignment(note_list, lyrics_list):
#     print(note_list[0], len(lyrics_list))    
    note_list_new = []
    lyrics_list_new = []
    i = j = 0
    while i < len(note_list) and j < len(lyrics_list):
        while not (note_list[i][1] == lyrics_list[j][1]):
            #print(i)
            lyrics_list_new.append(lyrics_list[j][2])
            note_list_new.append(int(note_list[i][2]))
            i += 1
            if i >= len(note_list):
                break
         
        if i >= len(note_list) or j>= len(lyrics_list):
            break
            
            
        note_list_new.append(int(note_list[i][2]))
        lyrics_list_new.append(lyrics_list[j][2])
        i += 1 
        j += 1

    if len(lyrics_list_new) or len(note_list_new) < 300:
        note_list_new = note_list_new + note_list_new
        lyrics_list_new = lyrics_list_new + lyrics_list_new
        
    if len(lyrics_list_new) or len(note_list_new) < 300:
        note_list_new = note_list_new + note_list_new
        lyrics_list_new = lyrics_list_new + lyrics_list_new
        
    if len(lyrics_list_new) or len(note_list_new) < 300:
        note_list_new = note_list_new + note_list_new
        lyrics_list_new = lyrics_list_new + lyrics_list_new

    note_list_new = note_list_new[:300]
    lyrics_list_new = lyrics_list_new[:300]
    #print(len(note_list_new))
    return note_list_new, lyrics_list_new

In [338]:
original_data = []
#index = 0
for data1 in data:

    note,lyrics = alignment(data1[0],data1[1])

    # word embedding model
    from gensim.models import Word2Vec
    model = Word2Vec(np.array(data1[1])[:,2].tolist(), min_count=1,size = 50)

    #embed to get list
#     print(index)
    #index += 1
    
    for i in lyrics:
        s = str(i)
        n = ord(s[0])
        if n > 96 and n < 123 or n >64 and n < 91:
            #print(i)
            j = lyrics[lyrics.index(i)-1]
            lyrics[lyrics.index(i)] = j
            
            
    lyrics_song = [model.wv[i] for i in lyrics]
    note_song =[i for i in note]

    #turn list into tensor and unsqueeze tensor
    #print(np.array(lyrics_song).shape)
    tensor1 = torch.tensor(np.array(lyrics_song).reshape(300, -1))
   
    tensor2 = torch.tensor(np.array(note).reshape(300,-1))
   

    original_data.append([tensor1, tensor2, 1])

In [334]:
# tensor_combined = torch.FloatTensor(np.array(original_data))

class combined_data(Dataset):

    def __init__(self):
        # Initialize data, download, etc.
        # read with numpy or pandas
#         xy = np.loadtxt('./data/wine/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = len(original_data)

        # here the last column is the class label, the rest are the features
        self.x_data1 = original_data[:][0] # size [n_samples, n_features]
        self.x_data2 = original_data[:][1]
        self.y_data = original_data[:][2] # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data1[index], self.x_data2[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


# create dataset
dataset = combined_data()

# get first sample and unpack
first_data = dataset[0]
feature1, feature2, labels = first_data
print(feature1, feature2, labels)

# Load whole dataset with DataLoader
# shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses
# !!! IF YOU GET AN ERROR DURING LOADING, SET num_workers TO 0 !!!
train_loader = DataLoader(dataset=dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=0)

# convert to an iterator and look at one random sample
# dataiter = iter(train_loader)
# data = dataiter.next()
# features, labels = data
# print(features, labels)

# Dummy Training loop
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)
print(total_samples, n_iterations)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        
        # here: 178 samples, batch_size = 4, n_iters=178/4=44.5 -> 45 iterations
        # Run your training process
        if (i+1) % 5 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}| Inputs {inputs.shape} | Labels {labels.shape}')



# train_dataset = torchvision.datasets.MNIST(root='./data', 
#                                            train=True, 
#                                            transform=torchvision.transforms.ToTensor(),  
#                                            download=True)

train_loader = DataLoader(dataset=train_dataset, 
                                           batch_size=3, 
                                           shuffle=True)

# look at one random sample
dataiter = iter(train_loader)
data = dataiter.next()
inputs, targets = data
print(inputs.shape, targets.shape)

tensor([[ 1.6490e-03,  5.3956e-03, -3.4479e-03,  ..., -9.6215e-03,
          6.0410e-03,  7.1533e-03],
        [-9.6902e-03, -3.6995e-03, -6.0621e-03,  ..., -3.3347e-03,
         -8.0594e-04,  3.6852e-03],
        [-3.3844e-03,  6.9429e-04, -1.3131e-03,  ..., -5.8848e-05,
         -4.0899e-03,  1.6930e-04],
        ...,
        [-2.4842e-04,  5.1262e-03,  7.3300e-04,  ...,  1.3601e-03,
         -2.1284e-03, -3.7579e-05],
        [ 8.9685e-03,  6.0215e-03,  7.8130e-04,  ..., -6.4805e-03,
          8.1382e-05, -7.6101e-03],
        [ 5.9058e-03,  8.3611e-03,  3.8252e-03,  ...,  7.7989e-03,
         -5.0266e-04,  4.7592e-03]]) tensor([[ 1.4713e-03,  8.8400e-03,  2.5835e-04,  ..., -3.9549e-03,
         -9.6205e-03,  4.3546e-03],
        [-9.1384e-03,  7.8891e-03,  7.0957e-03,  ...,  7.8841e-05,
         -5.1244e-03, -5.5614e-03],
        [-3.1716e-03,  2.0708e-03,  4.0035e-03,  ...,  9.5178e-03,
          1.3263e-04, -9.9696e-03],
        ...,
        [-7.4650e-03, -3.2659e-03,  1.8810e-04

IndexError: list index out of range